In [ ]:
!pip install datasets
!pip install -q -U PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 342 kB 10.1 MB/s 
     |████████████████████████████████| 84 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 18.5 MB/s 
     |████████████████████████████████| 212 kB 15.6 MB/s 
     |████████████████████████████████| 136 kB 43.7 MB/s 
     |████████████████████████████████| 127 kB 43.0 MB/s 
     |████████████████████████████████| 271 kB 42.9 MB/s 
     |████████████████████████████████| 144 kB 52.0 MB/s 
     |████████████████████████████████| 94 kB 562 kB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import pandas as pd
from datasets import load_metric
import torch
!pip install rouge_score
!pip install transformers

bleu = load_metric('bleu')
rouge = load_metric('rouge')

     |████████████████████████████████| 4.2 MB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 15.5 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
data_file = drive.CreateFile({'id': '1kWKsTWPp3xfSqNjOfx85rzGzktArq8sU'})
data_file.GetContentFile('test_out.csv')
test_data = pd.read_csv('test_out.csv')

In [ ]:
test_data.head()

,Unnamed: 0,Unnamed: 0.1,label,text,paraphrase,style_example
0,0,0,3,The imprisonment of Grotius was not the worst ...,Grotius was not the worst of all.,"And yet--and yet, he couldn't forbid a little ..."
1,1,1,7,The unfettered sun takes his unbounded reign,the unfettered sun is free to reign,If he's amang his friends or foes?
2,2,2,0,lol srry fun question but keep your head up do...,"excuse me, but I'm sorry, but I'm sorry, but I'm",Today just ain't my day .
3,3,3,3,"I tried to speak, but could not . ""","I'm trying to talk, but I can't.","How I forgot everything then--my mother, my dr..."
4,4,4,7,"Dancing upon the waves, as if to please","dancing on the waves, as if they were happy","And O, her een, they spak sic things!"


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, BertModel

style_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 11,
    output_attentions = False,
)


data_file = drive.CreateFile({'id': '1pJjclb_Ht-fklPtK7baDj1dvdm2U8Cub'})
data_file.GetContentFile('style_classifier.pth')

style_model.load_state_dict(torch.load('style_classifier.pth'))

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: ignored

In [ ]:

from datasets import Metric, MetricInfo, Features, Sequence, Value
import numpy as np

class accMetric():
  def __init__(self, model):
    self.model = model

  def compute(self, predictions, labels):
    outputs = []
    n = 100
    for i in range(0, len(predictions), n):
        p = predictions[i:i + n]
        input_ids, attn_masks = self.tokenize_and_format(p)
        outputs.extend(self.get_label_output(input_ids, attn_masks))
    labels = np.array(labels)
    outputs = np.array(outputs)
    N = labels.shape[0]
    accuracy = (labels == outputs).sum() / N
    return accuracy

  def get_label_output(self, input_ids, masks):
    self.model.eval()

    input_id_tensors = torch.stack([data[0] for data in input_ids])
    input_mask_tensors = torch.stack([data[0] for data in masks])

    with torch.no_grad():        
          outputs = self.model(input_id_tensors, 
                          attention_mask=input_mask_tensors)
          outputs = np.argmax(np.array(outputs.logits), axis=1)
  
    return outputs

  def tokenize_and_format(self, sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      
                            add_special_tokens = True, 
                            max_length = 64,           
                            padding = 'max_length',
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt', 
                      )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    return input_ids, attention_masks

In [ ]:
content_baseline = test_data[["text", "text"]]
style_baseline = test_data[["text", "style_example"]]

content_baseline.set_axis(["text", "output"], axis=1, inplace=True)
style_baseline.set_axis(["text", "output"], axis=1, inplace=True)

class EvalPipeline():
  def __init__(self, predictions, references):
    self.bleu = load_metric('bleu')
    self.rouge = load_metric('rouge')
    self.predictions = predictions
    self.references = [references]
  
  def run(self):
    
      


SyntaxError: ignored

In [ ]:
#acc = accMetric(style_model)
splt = lambda x: [y.split(' ') for y in x]
splt_ref = lambda x: [[y.split(' ')] for y in x]
p = test_data.head().text.values
print(p)

#acc.compute(p, test_data.head(600).label.values)
for x in p:
  bleu_scores = bleu.compute(predictions=[x.split(' ')],references=[[x.split(' ')]])
  print(bleu_scores)


['The imprisonment of Grotius was not the worst that now befell him.'
 'The unfettered sun takes his unbounded reign'
 'lol srry fun question but keep your head up dont cry'
 'I tried to speak, but could not . "'
 'Dancing upon the waves, as if to please']
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 12, 'reference_length': 12}
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 7, 'reference_length': 7}
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 11, 'reference_length': 11}
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 9, 'reference_length': 9}
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 8, 'reference_length': 8}
